In [75]:
import sys
import json
import os
from src.data import generate_dataset
from src.data import clear
import pandas as pd
import subprocess
from twarc import Twarc
from datetime import datetime, date, timedelta
import run
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
def configure_twarc(api_keys_json):
    with open(api_keys_json) as f:
        keys = json.load(f)
        t = Twarc(
            keys['consumer_key'],
            keys['consumer_secret'],
            keys['access_token'],
            keys['access_token_secret']
        )
    return t

In [2]:
def sample(df, sample_rate, id_column):
    return df.iloc[::sample_rate, :][id_column]
def sample_files(raw_data_path, sample_rate, dehydrated_sample_path, id_column):
    if not os.path.exists(dehydrated_sample_path):
        os.makedirs(dehydrated_sample_path)
    # find the filenames 
    file_names = sorted([name for name in os.listdir(raw_data_path) if 'dataset' in name])
    # for every .tsv under the directory
    for file in file_names:
        # read the file into df
        df = pd.read_table(f'{os.path.join(raw_data_path, file)}')
        # sample it
        a_sample = sample(df, sample_rate, id_column)
        # get the saving file name {original_file_name}.txt 
        fname = file.split('.')[0] + '.txt'
        print(f'sampling for dataset on {fname}')
        # save the sample to the path
        a_sample.to_csv(os.path.join(dehydrated_sample_path, fname), index = False, header = None)

In [3]:
# helper function
def obliviate(path):
    for fname in os.listdir(path):
        print(f'deleting{fname} under {path}')
        file_path = os.path.join(path, fname)
        try:
            if os.path.isfile(file_path):
                # ELDRITCH BLAST!!!!
                os.remove(file_path)
            elif os.path.isdir(file_path):
                # ELDRITCH BLAST!!!!
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# clear script
def clean(paths):
    for path in paths:
        obliviate(path)
    

In [69]:
# Get the information from raw tweets we just obtained
# processed_data_path is the path for the sampled dehydrated ids
def rehydrate_tweets(raw_data_path, processed_data_path, project_path, json_data_path, sample_rate, id_column, api_keys_json):
    # Sample data and write to processed_data_path
#     sample_files(raw_data_path, sample_rate, processed_data_path, id_column)
    
    t = configure_twarc(api_keys_json)
    # Rehydrate text file
    if not os.path.exists(json_data_path):
        os.makedirs(json_data_path)
        
    sample_names = set([name.split('.')[0] for name in os.listdir(processed_data_path)])
    json_names = set([name.split('.')[0] for name in os.listdir(json_data_path)])
    missing_names = sample_names - json_names
    print(f'here are the missing jsons: {missing_names}')
        
    for file in sorted(missing_names):
        # absolute path for txt id file
        abs_path = project_path + os.path.join(processed_data_path, file + '.txt')
        # absolute path for target directory
        name = file + '.jsonl'
        abs_target_path = project_path + json_data_path + name
        print(f'saving to {abs_target_path}')
        
        with open(abs_target_path, 'w') as outfile:
            for tweet in t.hydrate(abs_path):
                outfile.write(tweet)
                outfile.write('\n')


In [5]:
with open('./config/data_params.json') as f:
    data_params = json.load(f)
with open('./config/sample_params.json') as f:
    sample_params = json.load(f)

# Cfg variables
raw_data_path = data_params['raw_data_path']
processed_data_path = data_params['dehydrated_data_path']
project_path = data_params['absolute_project_path']
twarc_path = data_params['twarc_path']
rehydrated_json_path = data_params['rehydrated_json_path']
id_column = data_params['id_column']
from_day = data_params['from_day']
to_day = data_params['to_day']
want_cleaned = data_params['want_cleaned']
api_keys_path = data_params['api_keys']

sample_rate = sample_params['sample_every']


In [73]:
rehydrate_tweets(raw_data_path, processed_data_path, project_path, rehydrated_json_path, sample_rate, id_column, api_keys_path)

FileNotFoundError: [Errno 2] No such file or directory: 'data/temp/sample_by_date/'

In [6]:
sample_names = set([name.split('.')[0] for name in os.listdir(processed_data_path)])
json_names = set([name.split('.')[0] for name in os.listdir(rehydrated_json_path)])
missing_names = sample_names - json_names
sample_filename = [name + '.txt' for name in missing_names]

In [8]:
# generate_dataset.download_latest_datasets(raw_data_path, from_day, to_day, want_cleaned)

In [9]:
# generate_dataset.rehydrate_tweets(raw_data_path, processed_data_path, project_path, rehydrated_json_path, sample_rate, id_column, twarc_path)

In [76]:
run.main()

FileNotFoundError: [Errno 2] No such file or directory: '../shared_dir'